# Tweet Emotions Analysis (12 emotions) <a class="anchor" id="tea"></a>

<a href="https://www.linkedin.com/in/ouassim-adnane/">Ouassim Adnane</a> 08 June 2020

# Overview 

In this notebook, I've used a tweets dataset that contains tweet text with 12 emotions (neutral, worry, happiness, sadness, love, surprise, fun, relief, hate, empty, enthusiasm, boredom and anger) and the goal is to predict the percentage of emotions in a giving text

To achieve that goal I've used some techniques fist to preprocess the text data :

<li>correct misspelled text</li>
<li>replace English contractions with there meaning (isn't => is not)</li>
<li>remove some punctuations, URLS user mentions and extra spaces</li>
<li>replace emojis with there meaning</li><br>

For the modeling part I've used LSTM's and Roberta base Model:
<li>First a Basic LSTM </li>
<li>LSTM model with glove word embeddings</li>
<li>Roberta Base model </li>
<br>
In the final part, I've made a donut chart that detects the level of emotions is a particular text.

# Load Necesarry Packages <a class="anchor" id="tu"></a>


In [1]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null
!pip install emoji KaggleDatasets transformers

     |████████████████████████████████| 133kB 9.2MB/s 
     |████████████████████████████████| 2.1MB 8.4MB/s 
     |████████████████████████████████| 901kB 43.8MB/s 
     |████████████████████████████████| 3.3MB 51.9MB/s 


In [2]:
import preprocessor as p
import numpy as np 
import pandas as pd 
import emoji
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm

# Data preparation / Load Datasets  <a class="anchor" id="dp"></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Create Sentiment train dataset with text and labels
sentiment_train_text = pd.read_csv("/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/train_text.txt", 
                              delimiter="\t", names=["Tweet text"])
sentimemt_train_labels = pd.read_csv("/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/train_labels.txt",
                                delimiter="\t", names=["labels"])
df_sentiment_train = sentiment_train_text.join(sentimemt_train_labels)

In [5]:
#Create Sentiment validation dataset with text and labels
sentiment_val_text = pd.read_csv("/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/val_text.txt", 
                              delimiter="\t", names=["Tweet text"])
sentimemt_val_labels = pd.read_csv("/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/val_labels.txt",
                                delimiter="\t", names=["labels"])
df_sentiment_val = sentiment_val_text.join(sentimemt_val_labels)

In [6]:
#Create Sentiment test dataset with text and labels
sentiment_test_text = pd.read_csv("/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/test_text.txt", 
                              delimiter="\t", names=["Tweet text"])
sentimemt_test_labels = pd.read_csv("/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/test_labels.txt",
                                delimiter="\t", names=["labels"])
df_sentiment_test = sentiment_test_text.join(sentimemt_test_labels)

In [7]:
#Create function view content in .txt file
def text_file_reader(data):
  with open(data) as f:
    contents = f.read()
    print(contents)

In [8]:
#Types of labels for sentiment dataset
text_file_reader('/content/drive/MyDrive/CE888 Data Science for Decision Making/Tweet Evaluation Project/sentiment/mapping.txt')

0	negative
1	neutral
2	positive



# Modeling  <a class="anchor" id="m"></a>


### Encoding the data and train test split <a class="anchor" id="m-ed"></a>


In [9]:
sent_to_id  = {"negative":0, "neutral":1,"positive":2}

In [10]:
def encoder(y_label):
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(y_label)

  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  Y = onehot_encoder.fit_transform(integer_encoded)
  return Y

In [11]:
X_train = df_sentiment_train['Tweet text']
y_train = encoder(df_sentiment_train['labels'])

X_test = df_sentiment_test['Tweet text']
y_test = encoder(df_sentiment_test['labels'])

X_val = df_sentiment_val['Tweet text']
y_val = encoder(df_sentiment_val['labels'])

In [12]:
len(X_train)

36373

### LSTM <a class="anchor" id="m-l"></a>

In [13]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 160
Epoch = 5
token.fit_on_texts(list(X_train) + list(X_test))
X_train_pad = sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=max_len)
X_val_pad = sequence.pad_sequences(token.texts_to_sequences(X_val), maxlen=max_len)
X_test_pad = sequence.pad_sequences(token.texts_to_sequences(X_test), maxlen=max_len)

In [14]:
w_idx = token.word_index

In [15]:
embed_dim = 160
lstm_out = 250

model = Sequential()
model.add(Embedding(len(w_idx) +1 , embed_dim,input_length = X_test_pad.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.core.Dense(3, activation='softmax')) #Number of Classification
#adam rmsprop 
model.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 160, 160)          9080160   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 160, 160)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 250)               411000    
_________________________________________________________________
dense (Dense)                (None, 3)                 753       
Total params: 9,491,913
Trainable params: 9,491,913
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
batch_size = 32

In [17]:
Epoch = 10
model.fit(X_train_pad, y_train, epochs = Epoch, batch_size=batch_size,validation_data=(X_val_pad, y_val))

Epoch 1/10
1137/1137 [==============================] - 1349s 1s/step - loss: 1.0221 - accuracy: 0.4462 - val_loss: 1.0213 - val_accuracy: 0.4393
Epoch 2/10
1137/1137 [==============================] - 1344s 1s/step - loss: 0.9771 - accuracy: 0.5184 - val_loss: 1.0729 - val_accuracy: 0.4134
Epoch 3/10
1137/1137 [==============================] - 1341s 1s/step - loss: 0.7073 - accuracy: 0.6948 - val_loss: 1.2383 - val_accuracy: 0.4153
Epoch 4/10
1137/1137 [==============================] - 1341s 1s/step - loss: 0.4591 - accuracy: 0.8084 - val_loss: 1.5781 - val_accuracy: 0.4110
Epoch 5/10
1137/1137 [==============================] - 1341s 1s/step - loss: 0.3022 - accuracy: 0.8770 - val_loss: 1.8409 - val_accuracy: 0.3931
Epoch 6/10
1137/1137 [==============================] - 1341s 1s/step - loss: 0.2036 - accuracy: 0.9181 - val_loss: 2.3358 - val_accuracy: 0.4067
Epoch 7/10
1137/1137 [==============================] - 1340s 1s/step - loss: 0.1560 - accuracy: 0.9372 - val_loss: 2.6544 -

In [18]:
# Clone trained LSTM model by creating a new object
LSTM = model

# Evaluate performance on test dataset
Epoch = 1
LSTM.fit(X_test_pad, y_test, epochs = Epoch, batch_size=batch_size,validation_data=(X_val_pad, y_val))

376/376 [==============================] - 433s 1s/step - loss: 1.1040 - accuracy: 0.4679 - val_loss: 1.1469 - val_accuracy: 0.4091


#### Test LSTM on texts <a class="anchor" id="m-lr"></a>


In [19]:
def get_sentiment(model,text):
    twt = token.texts_to_sequences([text])
    twt = sequence.pad_sequences(twt, maxlen=max_len, dtype='int32')
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [20]:
def plot_result(df):
    #colors=['#D50000','#000000','#008EF8','#F5B27B','#EDECEC','#D84A09','#019BBD','#FFD000','#7800A0','#098F45','#807C7C','#85DDE9','#F55E10']
    #fig = go.Figure(data=[go.Pie(labels=df.sentiment,values=df.percentage, hole=.3,textinfo='percent',hoverinfo='percent+label',marker=dict(colors=colors, line=dict(color='#000000', width=2)))])
    #fig.show()
    colors={'negative':'rgb(213,0,0)','neutral':'rgb(0,0,0)',
                    'positive':'rgb(0,142,248)'}
    col_2={}
    for i in result.sentiment.to_list():
        col_2[i]=colors[i]
    fig = px.pie(df, values='percentage', names='sentiment',color='sentiment',color_discrete_map=col_2,hole=0.3)
    fig.show()

In [22]:
result =get_sentiment(model,"Had an absolutely brilliant day ðŸ˜ loved seeing an old friend and reminiscing")
plot_result(result)

1/1 - 0s


## Albert Base

In [22]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        #return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

def build_model(transformer, max_len=160):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(3, activation='softmax')(cls_token) #Number of Classification
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [23]:
AUTO = tf.data.experimental.AUTOTUNE
MODEL = 'albert-base-v2'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
X_train_t = regular_encode(X_train.to_list(), tokenizer, maxlen=max_len)
X_test_t = regular_encode(X_test.to_list(), tokenizer, maxlen=max_len)
X_val_t = regular_encode(X_val.to_list(), tokenizer, maxlen=max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



In [24]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train_t, y_train))
    .repeat()
    .shuffle(1995)
    .batch(batch_size)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_val_t, y_val))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)

In [25]:
transformer_layer = TFAutoModel.from_pretrained(MODEL)
albert = build_model(transformer_layer, max_len=max_len)
albert.summary()

Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [26]:
n_steps = X_train.shape[0] // batch_size
Epoch = 10
albert.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=Epoch)

Epoch 1/10
1136/1136 [==============================] - 2218s 2s/step - loss: 1.0658 - accuracy: 0.4285 - val_loss: 1.0245 - val_accuracy: 0.4017
Epoch 2/10
1136/1136 [==============================] - 2185s 2s/step - loss: 1.0266 - accuracy: 0.4291 - val_loss: 1.0287 - val_accuracy: 0.4362
Epoch 3/10
1136/1136 [==============================] - 2187s 2s/step - loss: 1.0222 - accuracy: 0.4431 - val_loss: 1.0311 - val_accuracy: 0.4393
Epoch 4/10
1136/1136 [==============================] - 2208s 2s/step - loss: 1.0246 - accuracy: 0.4336 - val_loss: 1.0208 - val_accuracy: 0.4393
Epoch 5/10
1136/1136 [==============================] - 2188s 2s/step - loss: 1.0224 - accuracy: 0.4371 - val_loss: 1.0243 - val_accuracy: 0.4393
Epoch 6/10
1136/1136 [==============================] - 2184s 2s/step - loss: 1.0188 - accuracy: 0.4401 - val_loss: 1.0277 - val_accuracy: 0.4393
Epoch 7/10
1136/1136 [==============================] - 2188s 2s/step - loss: 1.0224 - accuracy: 0.4341 - val_loss: 1.0354 -

In [27]:
# Clone trained albert model by creating a new object
albert2 = albert

# Modify test dataset to fit into the model
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test_t, y_test))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)

# Evaluate performance on test dataset
Epoch = 1
albert2.fit(test_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=Epoch)

1136/1136 [==============================] - 745s 654ms/step - loss: 1.0414 - accuracy: 0.4824 - val_loss: 1.1530 - val_accuracy: 0.4393


### Test ALBERT on Texts

In [28]:
def get_sentiment2(model,text):
    x_test1 = regular_encode([text], tokenizer, maxlen=max_len)
    test1 = (tf.data.Dataset.from_tensor_slices(x_test1).batch(1))
    #test1
    sentiment = model.predict(test1,verbose = 0)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [29]:
result =get_sentiment2(albert,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
plot_result(result)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).

